# SAM: basi del formato e metrica di mapping

Obiettivi:
- Capire la struttura di un file SAM (header + righe di allineamento).
- Leggere le colonne principali: QNAME, FLAG, RNAME, POS, MAPQ, CIGAR.
- Calcolare qualche metrica di QC: % mapped, distribuzione MAPQ.

Useremo un piccolo SAM **sintetico** definito direttamente nel notebook.

In [ ]:
from __future__ import annotations
import matplotlib.pyplot as plt

%matplotlib inline

## Piccolo SAM sintetico

- 3 righe di header (`@SQ`, `@PG`)
- 8 read (alcuni mappati, uno unmapped, alcuni in pair corretti)

In [ ]:
sam_text = """@SQ\tSN:chr1\tLN:100000
@SQ\tSN:chr2\tLN:80000
@PG\tID:aligner\tPN:BWA-MEM\tVN:0.7.17
READ1\t99\tchr1\t1000\t60\t100M\t=\t1100\t200\tACGT...\tIIII...
READ1\t147\tchr1\t1100\t60\t100M\t=\t1000\t-200\tACGT...\tIIII...
READ2\t83\tchr1\t5000\t42\t100M\t=\t5200\t200\tACGT...\tHHHH...
READ2\t163\tchr1\t5200\t40\t100M\t=\t5000\t-200\tACGT...\tHHHH...
READ3\t0\tchr2\t3000\t10\t50M50S\t*\t0\t0\tACGT...\tFFFF...
READ4\t16\tchr2\t10000\t0\t100M\t*\t0\t0\tACGT...\t!!!!...
READ5\t4\t*\t0\t0\t*\t*\t0\t0\tACGT...\tIIII...
READ6\t99\tchr1\t20000\t55\t100M\t=\t20120\t220\tACGT...\tIIII...
"""

## Parsing delle righe SAM

In [ ]:
def parse_sam(text: str):
    """Restituisce un elenco di record (dict) per le righe NON di header."""
    records = []
    for line in text.strip().splitlines():
        if line.startswith("@"):  # header
            continue
        fields = line.split("\t")
        qname = fields[0]
        flag = int(fields[1])
        rname = fields[2]
        pos = int(fields[3])
        mapq = int(fields[4])
        cigar = fields[5]
        rnext = fields[6]
        pnext = int(fields[7])
        tlen = int(fields[8])
        seq = fields[9]
        qual = fields[10]
        records.append(
            {
                "qname": qname,
                "flag": flag,
                "rname": rname,
                "pos": pos,
                "mapq": mapq,
                "cigar": cigar,
                "rnext": rnext,
                "pnext": pnext,
                "tlen": tlen,
                "seq": seq,
                "qual": qual,
            }
        )
    return records

records = parse_sam(sam_text)
print(f"Numero di allineamenti: {len(records)}")
records[0]

## Interpretare il FLAG

Il campo FLAG è un intero con bit diversi (somma di potenze di 2).
Vediamo alcune maschere comuni:

- 0x1: read è paired
- 0x2: read è in un "proper pair"
- 0x4: read unmapped
- 0x8: mate unmapped
- 0x10: read allineato sullo strand complementare
- 0x20: mate sullo strand complementare

In [ ]:
FLAG_BITS = {
    0x1: "paired",
    0x2: "proper_pair",
    0x4: "read_unmapped",
    0x8: "mate_unmapped",
    0x10: "read_reverse_strand",
    0x20: "mate_reverse_strand",
}

def decode_flag(flag: int):
    return [name for bit, name in FLAG_BITS.items() if flag & bit]

for rec in records:
    print(rec["qname"], rec["flag"], decode_flag(rec["flag"]))

## Metrica: % read mappate vs unmappate

In [ ]:
total = len(records)
unmapped = sum(1 for r in records if r["flag"] & 0x4)
mapped = total - unmapped

print(f"Totale read: {total}")
print(f"Mapped     : {mapped} ({mapped/total*100:.1f}%)")
print(f"Unmapped   : {unmapped} ({unmapped/total*100:.1f}%)")

## Distribuzione della mapping quality (MAPQ) per i read mappati

In [ ]:
mapq_values = [r["mapq"] for r in records if not (r["flag"] & 0x4)]

print("MAPQ values:", mapq_values)

plt.hist(mapq_values, bins=range(0, 65, 5), align="left", rwidth=0.8)
plt.xlabel("MAPQ")
plt.ylabel("Conteggio read")
plt.title("Distribuzione MAPQ (dataset sintetico)")
plt.show()

In dati reali:
- molte read con MAPQ molto basso possono indicare problemi di allineamento
  (genoma di riferimento sbagliato, regioni molto ripetitive, contaminazione, ...)

## Paired-end e insert size (TLEN)

Guardiamo solo i read in proper pair (`FLAG` include 0x2) e TLEN != 0.

In [ ]:
proper_pairs = [
    r for r in records
    if (r["flag"] & 0x2) and r["tlen"] != 0 and not (r["flag"] & 0x4)
]

for r in proper_pairs:
    print(
        r["qname"],
        "rname:", r["rname"],
        "pos:", r["pos"],
        "TLEN:", r["tlen"],
    )

In un BAM reale potremmo fare l'istogramma degli insert size (|TLEN|) per valutare se
la distribuzione è coerente con il design della libreria.